In [ ]:
# Import Libraries
from pandas.api.types import is_numeric_dtype
from collections import defaultdict
import plotly.graph_objects as go
from Bio import Entrez, Medline
import matplotlib.pyplot as plt
from pathlib import Path
import pubchempy as pcp
from tqdm import tqdm
import networkx as nx
import pandas as pd
import numpy as np
import requests
import signal
import time
import re

# Set email address (required for NCBI API usage)
Entrez.email = 'sks6765@psu.edu'

In [ ]:
def create_graph_viz(dataframe, column1, column2, label=1):
    # Creating an interactive graph
    G = nx.Graph()
    
    # Add edges to the graph
    for index, row in dataframe.iterrows():
        G.add_edge(row[column1], row[column2], label=row[label] if label != 1 else 1)
    
    # Add edge labels
    edge_labels = {(u, v): d['label'] for u, v, d in G.edges(data=True)}
    nx.set_edge_attributes(G, edge_labels, 'label')
    
    # Position nodes using spring layout
    pos = nx.spring_layout(G)
    
    # Extract node and edge information
    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]
    edge_x = []
    edge_y = []
    edge_text = []
    for edge in G.edges():
        source, target = edge
        x0, y0 = pos[source]
        x1, y1 = pos[target]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
        edge_text.append(G.get_edge_data(*edge)['label'])  # Use existing edge labels
    
    # Create a Plotly figure
    fig = go.Figure(data=[go.Scatter(
        x=edge_x,
        y=edge_y,
        mode='lines',
        line_shape='spline',
        opacity=0.5,
        hoverinfo='none'
    ),
    go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers',
        hoverinfo='text',
        hovertext=[f'Node {node}' for node in G.nodes()]
    ),
    go.Scatter(
        x=[(pos[edge[0]][0] + pos[edge[1]][0]) / 2 for edge in G.edges()],
        y=[(pos[edge[0]][1] + pos[edge[1]][1]) / 2 for edge in G.edges()],
        mode='text',
        text=edge_text,
        textposition='middle center',
        hoverinfo='none'
    )])
    
    # Customize the layout
    fig.update_layout(
        showlegend=False,
        hovermode='x',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
    
    # Display the figure
    fig.show()

### DDI Sampling

In [ ]:
ddi = pd.read_csv("bio-decagon-combo.csv")

In [ ]:
class TimeoutError(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutError

def CID_information(CID: int, timeout: int = 5):

    # Set the signal handler
    signal.signal(signal.SIGALRM, timeout_handler)

    # Set the alarm
    signal.alarm(timeout)

    try:
        cleaned_cid = int(re.sub(r"CID[m|s]*0*", "", CID))
        compound = pcp.Compound.from_cid(cleaned_cid)
        try:
            compound_name = compound.synonyms[0]
        except:
            compound_name = compound.iupac_name
        return (compound_name, compound.canonical_smiles)
    except TimeoutError:
        print("Timeout: Function took too long to complete.")
        return None
    finally:
        # Disable the alarm
        signal.alarm(0)

In [ ]:
random_state=85
np.random.seed(random_state)

# Creating the graph from the ddi dataframe
G = nx.from_pandas_edgelist(ddi, 'STITCH 1', 'STITCH 2')

# Get the largest connected component
largest_component = max(nx.connected_components(G), key=len)

# Sample nodes from the largest component
sample_size = 300
sample_nodes = np.random.choice(list(largest_component), size=sample_size, replace=False)

# Get the subgraph induced by the sampled nodes
subgraph = G.subgraph(sample_nodes)

# Convert the subgraph back to a DataFrame
sampled_df = nx.to_pandas_edgelist(subgraph).sample(n=2000, random_state=random_state)

sampled_df.rename(columns={"source": "STITCH 1", "target": "STITCH 2"}, inplace=True)

# Merging the sampled df with base ddi to get the side effect names
sampled_df = pd.merge(sampled_df, ddi).drop_duplicates(subset=["STITCH 1", "STITCH 2"])

In [ ]:
# Sanity Check the output

# Visualizing sampled_df as a graph to see that there are no disconnected edges
create_graph_viz(sampled_df, "STITCH 1", "STITCH 2")

all_drugs_according_to_df = len(set(sampled_df['STITCH 1'].to_list()).union(set(sampled_df['STITCH 2'].to_list())))

# Verify that you get the same value from the sampled_df as a graph
G = nx.from_pandas_edgelist(sampled_df, 'STITCH 1', 'STITCH 2')
number_of_drug_nodes = len(G.nodes())

assert all_drugs_according_to_df == number_of_drug_nodes

In [ ]:
def save_dataframe(all_information):
    pd.DataFrame(all_information, columns=["drug_1_CID", "drug_1_name", "drug_1_SMILES", "relationship", "drug_2_CID", "drug_2_name", 
                                               "drug_2_SMILES"]).to_csv("DDI_subset.csv", index=False)

number_of_ddis_to_retain = 300
all_information = []

for idx, sample in tqdm(enumerate(sampled_df.itertuples(index=False))):    
    drug_1_CID = sample[0]
    drug_2_CID = sample[1]
    relationship = sample[3]

    drug_1_info = CID_information(drug_1_CID)
    drug_2_info = CID_information(drug_2_CID)
    
    try:
        # I want the drug name to not include any numbers or brackets, i.e., only keep regular words.
        if (not re.search(r"[\W\d]|(sulfate)", drug_1_info[0])) and (not re.search(r"[\W\d]|(sulfate)", drug_2_info[0])):

            # This ensures that when I am retrieving the background information, I will get 20 pubmed hits.
            drug1_query = f"{drug_1_info[0]} AND hasabstract AND Humans AND (AD OR AE OR PK OR PD OR CO OR TU OR DE)"
            with Entrez.esearch(db='pubmed', term=drug1_query, retmax=20, sort="relevance") as handle:
                paper_list_drug1 = Entrez.read(handle)["IdList"]
    
            drug2_query = f"{drug_2_info[0]} AND hasabstract AND Humans AND (AD OR AE OR PK OR PD OR CO OR TU OR DE)"
            with Entrez.esearch(db='pubmed', term=drug2_query, retmax=20, sort="relevance") as handle:
                paper_list_drug2 = Entrez.read(handle)["IdList"]
            
            if len(paper_list_drug1) == 20 and len(paper_list_drug2) == 20:            
                all_information.append((drug_1_CID, drug_1_info[0], drug_1_info[1], relationship, drug_2_CID, drug_2_info[0], drug_2_info[1]))
    except:
        continue

    # timeout after 50 API calls.
    if (idx % 50 == 0) and (idx != 0):
        print(f"Samples collected so far: {len(all_information)}. Saving checkpoint and cooling down ...")
        save_dataframe(all_information)
        time.sleep(5)        
    
    if len(all_information) == number_of_ddis_to_retain:
        print("Necessary number of DDIs obtained... exiting")
        save_dataframe(all_information)
        break

### DPI Sampling

In [ ]:
ddi_subset = pd.read_csv("DDI_subset.csv")
dpi = pd.read_csv("bio-decagon-targets-all.csv")
actions_subset = pd.read_csv("action_subset.csv")

all_drugs_CID = set(ddi_subset["drug_1_CID"].to_list()).union(set(ddi_subset["drug_2_CID"].to_list()))

dpi_subset = dpi.query("STITCH in @all_drugs_CID")

# Converting PubChem CID to STITCH CIDs (singular element) (https://www.biostars.org/p/155342/)
dpi_subset.loc[:, 'STITCH'] = dpi_subset['STITCH'].str.replace('CID0', 'CIDs')

In [ ]:
'''
I'm doing these following steps to retain the largest connected subgraph & remove disconnected components.
'''

# Creating the graph from the ddi dataframe
G = nx.from_pandas_edgelist(dpi_subset, 'STITCH', 'Gene')

# Get the largest connected component
largest_component = max(nx.connected_components(G), key=len)

subgraph = G.subgraph(largest_component)
dpi_subset = nx.to_pandas_edgelist(subgraph)

condition = dpi_subset["source"].apply(lambda x: isinstance(x, (int, float)))
dpi_subset.loc[condition, ['source', 'target']] = dpi_subset.loc[condition, ['target', 'source']].values
dpi_subset.rename(columns={"source": "STITCH", "target":"Gene"}, inplace=True)

In [ ]:
search_list = []
for row in tqdm(dpi_subset.itertuples()):  
    # Retrieving protein data from the given gene ID
    url = f"https://string-db.org/api/json/get_string_ids?identifiers={row.Gene}&species=9606"
    try:
        response = requests.get(url).json()[0]
        string_id = (response["stringId"], response["preferredName"], response["annotation"], row.Gene)
    except:
        continue

    search_list.append((row.STITCH, string_id))

In [ ]:
target_pairs = list(zip([x[0] for x in search_list], [x[1][0] for x in search_list]))
r = actions_subset[actions_subset.apply(lambda x: (x["item_id_a"], x["item_id_b"]) in set(target_pairs), axis=1)]

search_df = pd.DataFrame({"cid": [x[0] for x in search_list], "protein_number": [x[1][0] for x in search_list], 
                          "protein_name": [x[1][1] for x in search_list], "protein_desc": [x[1][2] for x in search_list],
                          "gene": [x[1][3] for x in search_list]})

# Need to drop duplicates because of "a is acting" column. That is, the same entry can be repeated twice because of "a is acting"
r = pd.merge(r, search_df, left_on=["item_id_a", "item_id_b"], right_on=["cid", "protein_number"], 
             how="inner").drop(columns=["item_id_a", "item_id_b", "action", "a_is_acting", "score"]).drop_duplicates()

r["cid"] = r["cid"].str.replace("CIDs", "CID0") # Going back to PubChem CID to be compatible with other DDI
r.to_csv("DPI_subset.csv", index=False)